In [29]:
import sys
sys.path.append('..')

import os
import geopandas as gpd
import pandas as pd
from sqlalchemy import create_engine, text
import matplotlib.pyplot as plt

In [30]:
from config import RUTA_UNIDAD_ONE_DRIVE
from config import RUTA_LOCAL_ONE_DRIVE
from config import POSTGRES_UTEA

RUTA_COMPLETA = os.path.join(RUTA_UNIDAD_ONE_DRIVE, RUTA_LOCAL_ONE_DRIVE)
PATH_SHP_RECORRIDOS = RUTA_UNIDAD_ONE_DRIVE + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Pulverizacion\2025\SHP\RECORRIDOS.shp'
GDF_RECORRIDOS = gpd.read_file(PATH_SHP_RECORRIDOS)

In [31]:
def obtener_engine():
    return create_engine(
        f"postgresql+psycopg2://{POSTGRES_UTEA['USER']}:{POSTGRES_UTEA['PASSWORD']}@{POSTGRES_UTEA['HOST']}:{POSTGRES_UTEA['PORT']}/{POSTGRES_UTEA['DATABASE']}"
    )

def obtener_parte_diario_sin_cod_labor():
    engine = obtener_engine()
    try:
        query = """
            SELECT DISTINCT idd, unidad_01, unidad_02
            FROM drones_pulverizacion.parte_diario_pulv 
            WHERE idd IS NOT NULL AND cod_labor IS NULL
        """
        # Usar pandas si no hay geometría involucrada
        df = pd.read_sql(query, engine)
        return df
    except Exception as e:
        print(f"❌ Error en la consulta: {e}")
        return pd.DataFrame()

def obtener_propiedad_por_idd(idd):
    engine = obtener_engine()
    try:
        query = f"""
            SELECT * FROM drones_pulverizacion.parte_diario_pulv WHERE idd= {idd}
        """
        gdf = gpd.read_postgis(query, engine, geom_col='geom')
        return gdf
    except Exception as e:
        print(f"❌ Error en la consulta: {e}")
        return gpd.GeoDataFrame()
    return None

def actualizar_area_rociada(id, val):
    engine = obtener_engine()
    try:
        with engine.begin() as conn:
            query = text("""
                UPDATE drones_pulverizacion.parte_diario_pulv
                SET 
                    area_rocia= :val
                WHERE id = :id
            """)
            conn.execute(query, {"id": id, "val": val})
            print(f"✔️ Area rociada actualizada de lote id: {id}")
    except Exception as e:
        print(f"❌ Error al actualizar: {e}")

def actualizar_faltantes_por_idd(idd, fecha_intermedia, fecha_min, fecha_max, numero_semana, num_vuelos, total_flight_time, total_caudal):
    engine = obtener_engine()
    try:
        with engine.begin() as conn:
            query = text("""
                UPDATE drones_pulverizacion.parte_diario_pulv
                SET 
                    fecha= :fecha_intermedia,
                    hora_ini= :fecha_min,
                    hora_fin= :fecha_max,
                    semana= :numero_semana,
                    num_vuelos= :num_vuelos,
                    horas_vuel= :total_flight_time,
                    total_caud= :total_caudal
                WHERE idd = :idd
            """)
            conn.execute(query, {
                "idd": idd, 
                "fecha_intermedia": fecha_intermedia,
                "fecha_min": fecha_min,
                "fecha_max": fecha_max,
                "numero_semana": numero_semana,
                "num_vuelos": num_vuelos,
                "total_flight_time": total_flight_time,
                "total_caudal": total_caudal,
            })
            print(f"✔️ Datos faltantes de idd: {idd} cargados correctamente")
    except Exception as e:
        print(f"❌ Error al actualizar: {e}")

In [33]:
propiedades_procesar = obtener_parte_diario_sin_cod_labor()
propiedades_procesar

,idd,unidad_01,unidad_02
0,75,1260,SAN JUAN DE MAROTAS--LIDIA DURAN


In [34]:
selec_recorridos

,file,fecha,hora,id,drone,ctrl_id,pilot,fl_time,mode,height,spacing,fl_speed,area,spray,idd,geometry,fl_time_sec
1733,T50-1_20251210091543_R2562597992.kml,2025-12-10 00:00:00,09:15:43,R2562597992,T50-1,1581F6BUB24630011R08,MARIO SANCHEZ,8:44,Auto,4.0,8.0,28.080001,2.232000,22.414,75,"POLYGON ((464207.505 8118198.426, 464207.079 8...",524
1734,T50-1_20251210093359_R2682733274.kml,2025-12-10 00:00:00,09:33:59,R2682733274,T50-1,1581F6BUB24630011R08,MARIO SANCHEZ,8:51,Auto,4.0,8.0,28.080001,2.230667,22.459,75,"POLYGON ((464212.671 8118176.915, 464212.666 8...",531
1735,T50-1_20251210095753_R2902981291.kml,2025-12-10 00:00:00,09:57:53,R2902981291,T50-1,1581F6BUB24630011R08,MARIO SANCHEZ,9:25,Auto,4.0,8.0,28.080001,2.645333,26.617,75,"POLYGON ((464231.533 8118140.184, 464230.944 8...",565
1736,T50-1_20251210101210_R3143251855.kml,2025-12-10 00:00:00,10:12:10,R3143251855,T50-1,1581F6BUB24630011R08,MARIO SANCHEZ,9:04,Auto,4.0,8.0,28.080001,2.558000,25.755,75,"POLYGON ((464243.407 8118110.730, 464242.898 8...",544
1737,T50-1_20251210103326_R3263387137.kml,2025-12-10 00:00:00,10:33:26,R3263387137,T50-1,1581F6BUB24630011R08,MARIO SANCHEZ,9:38,Auto,4.0,8.0,28.080001,2.686667,27.025,75,"POLYGON ((464255.076 8118081.076, 464254.558 8...",578
1738,T50-1_20251210104822_R3523680248.kml,2025-12-10 00:00:00,10:48:22,R3523680248,T50-1,1581F6BUB24630011R08,MARIO SANCHEZ,8:19,Auto,4.0,8.0,28.080001,2.128000,21.419,75,"POLYGON ((464267.545 8118051.693, 464266.070 8...",499
1739,T50-1_20251210110038_R3603770436.kml,2025-12-10 00:00:00,11:00:38,R3603770436,T50-1,1581F6BUB24630011R08,MARIO SANCHEZ,8:25,Auto,4.0,8.0,28.080001,2.166000,21.827,75,"POLYGON ((464278.705 8118021.928, 464278.619 8...",505
1740,T50-1_20251210111425_R3763950812.kml,2025-12-10 00:00:00,11:14:25,R3763950812,T50-1,1581F6BUB24630011R08,MARIO SANCHEZ,9:23,Auto,4.0,8.0,28.080001,2.656000,26.727,75,"POLYGON ((464290.921 8117992.657, 464287.483 8...",563
1741,T50-1_20251210112919_R4124356658.kml,2025-12-10 00:00:00,11:29:19,R4124356658,T50-1,1581F6BUB24630011R08,MARIO SANCHEZ,9:07,Auto,4.0,8.0,28.080001,2.595333,26.141,75,"POLYGON ((464302.103 8117962.609, 464301.871 8...",547
1742,T50-1_20251210114222_R4344604675.kml,2025-12-10 00:00:00,11:42:22,R4344604675,T50-1,1581F6BUB24630011R08,MARIO SANCHEZ,9:18,Auto,4.0,8.0,28.080001,2.704667,27.272,75,"POLYGON ((464319.791 8117918.204, 464319.648 8...",558


In [35]:
for i, row in propiedades_procesar.iterrows():
    idd = row.idd
    selec_recorridos = GDF_RECORRIDOS[GDF_RECORRIDOS['idd']==idd]
    if (len(selec_recorridos) == 0):
        continue
    # selccion de lotes planificados
    selec_plan = obtener_propiedad_por_idd(idd)
    # conviente en campo fecha a formato fecha
    selec_recorridos.loc[:, "fecha"] = pd.to_datetime(selec_recorridos["fecha"])
    # Obtener la fecha mínima y máxima
    fecha_min = selec_recorridos["fecha"].min()
    fecha_max = selec_recorridos["fecha"].max()
    fecha_intermedia = fecha_min + (fecha_max - fecha_min) / 2
    print(fecha_max)
    numero_semana = fecha_intermedia.isocalendar().week
    area_total_pulv = selec_recorridos['area'].sum()
    num_vuelos = len(selec_recorridos)
    total_caudal = selec_recorridos['spray'].sum()
    
    selec_recorridos = selec_recorridos.copy()
    
    # obtener horas de vuelo en formto 00:00:00
    selec_recorridos["fl_time_sec"] = selec_recorridos["fl_time"].apply(lambda x: int(x.split(":")[0]) * 60 + int(x.split(":")[1]))
    # Sum all seconds
    total_seconds = selec_recorridos["fl_time_sec"].sum()
    # Convert back to hh:mm:ss format
    hours = total_seconds // 3600
    minutes = (total_seconds % 3600) // 60
    seconds = total_seconds % 60
    total_flight_time = f"{hours:02}:{minutes:02}:{seconds:02}"  # 00:00:00

    # Aplicar el buffer a la geometría de los recorridod
    selec_recorridos["geometry"] = selec_recorridos.geometry.buffer(5.5)
    # Disolver todas las geometrías en un solo polígono
    buffer_dissolved = selec_recorridos.dissolve()
    # Calcular la intersección
    interseccion = gpd.overlay(buffer_dissolved, selec_plan, how="intersection")
    # Calcular el área de cada polígono de la intersección
    interseccion["area_rociado"] = interseccion.geometry.area / 10000
    # seleccion solo de la columnas de interes
    resumen = interseccion[['id_2', 'unidad_05', 'area_2', 'area_rociado']].copy()
    
    # PROCEDIMIENTO PARA AJUSTAR EL AREA DE ROCIADO
    suma_area_rociada = resumen['area_rociado'].sum()
    area_rociada_real = area_total_pulv  # aquí pon tu valor real
    # Factor inicial de ajuste
    factor_ajuste = area_rociada_real / suma_area_rociada
    # Ajuste proporcional inicial
    resumen['area_ajustada'] = resumen['area_rociado'] * factor_ajuste
    # Verificar que no exceda area_2 y corregir iterativamente si es necesario
    while any(resumen['area_ajustada'] > resumen['area_2']):
        exceso = resumen['area_ajustada'] > resumen['area_2']
        resumen.loc[exceso, 'area_ajustada'] = resumen.loc[exceso, 'area_2']
        # Recalcular factor solo con las filas no ajustadas al máximo permitido
        area_ajustada_parcial = resumen.loc[~exceso, 'area_ajustada'].sum()
        area_restante = area_rociada_real - resumen.loc[exceso, 'area_ajustada'].sum()
        # Evitar división por cero
        if area_ajustada_parcial == 0:
            break
        nuevo_factor = area_restante / area_ajustada_parcial
        resumen.loc[~exceso, 'area_ajustada'] *= nuevo_factor
    # calcular campos faltantes
    resumen["porcen_rociado"] = (resumen["area_ajustada"]) / (resumen["area_2"])
    resumen["dif_area"] = resumen["area_2"] - resumen["area_ajustada"]
    # recorrer resumen para actualizar area rociada en el parte diario
    for i, row in resumen.iterrows():
        print(row.id_2, row.area_ajustada)
        actualizar_area_rociada(row.id_2, row.area_ajustada)
    # cargar campos restantes del parte con respecto a idd
    actualizar_faltantes_por_idd(idd, fecha_intermedia, fecha_min, fecha_max, numero_semana, num_vuelos, total_flight_time, total_caudal)

2025-12-10 00:00:00
696 7.491157734014631
✔️ Area rociada actualizada de lote id: 696
693 9.179306752168682
✔️ Area rociada actualizada de lote id: 693
694 5.656369728998176
✔️ Area rociada actualizada de lote id: 694
695 9.012499274848514
✔️ Area rociada actualizada de lote id: 695
✔️ Datos faltantes de idd: 75 cargados correctamente
